In [0]:
# Initialize drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#Code to conntext: Adam
# Move to drive and import from onmt
%cd drive
%cd 'My Drive'
%cd AML

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Imports
from __future__ import absolute_import, division, print_function

import numpy as np
import os
import h5py
import re
import tensorflow as tf
import time
import math
import pickle

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Bidirectional,Dropout, Embedding, \
LSTM, Multiply, Lambda, Permute, Reshape, Masking, Input, Softmax, Subtract, \
Concatenate,Dropout,MaxPooling1D,AveragePooling1D,BatchNormalization, Maximum, Conv1D, Conv2D, \
MaxPooling2D, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import activations
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import Constant, RandomUniform
from tensorflow.contrib.layers import maxout

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Import developed modules
#from constants import *
from dataprep import *

Using TensorFlow backend.


In [0]:
# Initialize CNN constants
EMBEDDING = "" # CHOOSE EMBEDDING

CNN_DROPOUT = 0.2
CNN_R_DROPOUT = 0.1
CNN_BATCH_SIZE = 2
CNN_MAX_LENGTH = 35
N_TARGET=None
CNN_DATASET = "SST-2"
RUN_NUMBER = 10
BATCH_SIZE = 256
ENCODER_MODEL = 'LSTM'

if CNN_DATASET == "SST-5":
  elmoDir = "elmo-sst5"
  N_TARGET=5
elif CNN_DATASET == "SST-2":
  elmoDir = "elmo-sst2"
  N_TARGET=2

if EMBEDDING == "ELMO":
  CNN_BI_UNITS = 300
  FINAL_DIM = 1200
  CNN_DIM = 768
elif EMBEDDING == "COVE":
  CNN_BI_UNITS = 300
  FINAL_DIM = 1200
  CNN_DIM = 900
elif EMBEDDING == "GLOVE":
  CNN_BI_UNITS = 300
  FINAL_DIM = 1200
  CNN_DIM = 300
elif EMBEDDING == "CHAR+GLOVE":
  CNN_BI_UNITS = 300
  FINAL_DIM = 1200
  CNN_DIM = 400
elif EMBEDDING == "COVE+CHAR":
  CNN_BI_UNITS = 300
  FINAL_DIM = 1200
  CNN_DIM = 1000
elif EMBEDDING == "RANDOM":
  CNN_BI_UNITS = 300
  FINAL_DIM = 1200
  CNN_DIM = 300
elif EMBEDDING == "FRENCH":
  CNN_BI_UNITS = 300
  FINAL_DIM = 1200
  CNN_DIM = 900
elif EMBEDDING == "LARGE":
  CNN_BI_UNITS = 300
  FINAL_DIM = 1200
  CNN_DIM = 900
elif EMBEDDING == "BASELINE":
  CNN_BI_UNITS = 300
  FINAL_DIM = 1200
  CNN_DIM = 900

In [0]:
def read_sentiment_data(): 
  train_sent_path = os.path.join(elmoDir, 'dataset.txt')
  f_train_sent = open(train_sent_path, 'r', encoding='utf-8')
  train_sent = f_train_sent.read()
  f_train_sent.close()
  return train_sent

def read_sentiment_labels(): 
  labels = []
  train_sent_path = os.path.join(elmoDir, 'dataset_labels.txt')
  f_train_sent = open(train_sent_path, 'r', encoding='utf-8')
  for x in f_train_sent : 
    labels.append(float(x))
  f_train_sent.close()
  return labels

## Read Datasets for SST-2## 
def read_train_sentiment_data(): 
  train_sent_path = os.path.join(elmoDir, 'train_dataset.txt')
  f_train_sent = open(train_sent_path, 'r', encoding='utf-8')
  train_sent = f_train_sent.read()
  f_train_sent.close()
  return train_sent

def read_train_sentiment_labels(): 
  labels = []
  train_sent_path = os.path.join(elmoDir, 'train_dataset_labels.txt')
  f_train_sent = open(train_sent_path, 'r', encoding='utf-8')
  for x in f_train_sent : 
    labels.append(float(x))
  f_train_sent.close()
  return labels

def read_test_sentiment_data(): 
  train_sent_path = os.path.join(elmoDir, 'test_dataset.txt')
  f_train_sent = open(train_sent_path, 'r', encoding='utf-8')
  train_sent = f_train_sent.read()
  f_train_sent.close()
  return train_sent

def read_test_sentiment_labels(): 
  labels = []
  train_sent_path = os.path.join(elmoDir, 'test_dataset_labels.txt')
  f_train_sent = open(train_sent_path, 'r', encoding='utf-8')
  for x in f_train_sent : 
    labels.append(float(x))
  f_train_sent.close()
  return labels

def clean(data=None):
    data = re.sub('[0-9]+p*', 'n', data)  # replace all numbers with n
    data = re.sub('  ', ' ', data)  # remove double spaces
    data = re.sub("'", '', data)  # remove apostrophe
    data = data.split('\n')
    return data

  
if CNN_DATASET == "SST-5":

  # Read the SST data
  sent_labels = read_sentiment_labels()
  train_sent = read_sentiment_data()
  sent_labels = read_sentiment_labels()
  train_sent = clean(train_sent)

  tokenizer = Tokenizer(num_words=None, lower=True, oov_token='<UNK>')
  tokenizer.fit_on_texts(train_sent)

  # Tokenize with appropriate max_word length
  tokenizer = Tokenizer(num_words=len(tokenizer.word_counts.items()), lower=True, oov_token='<UNK>')
  tokenizer.fit_on_texts(train_sent)
  train_sent_tok = tokenizer.texts_to_sequences(train_sent)
  vocab = {k: v for k, v in tokenizer.word_counts.items() if v >= 1}
  vocab_len = len(vocab)

  # Max length in the SST training set was 50 
  train_sent_tok = pad_sequences(train_sent_tok, maxlen=CNN_MAX_LENGTH, truncating='post',
                            padding='post', value=0)

  i2w = {v: k for k, v in tokenizer.word_index.items()}
  # Create Glove Embedding dictionary
  glove_embedding_matrix = create_embedding_indexmatrix(vocab_len, 
                                                        embedding_dim=300,
                                                        dict_en=i2w)
else:
 
  # Read the SST data
  train_sent_labels = read_train_sentiment_labels()
  test_sent_labels = read_test_sentiment_labels()
  train_sent_data = read_train_sentiment_data()
  test_sent_data = read_test_sentiment_data()

  ## Combine Train sent data and test data for tokenization
  train_sent = train_sent_data + test_sent_data
  sent_labels= train_sent_labels + test_sent_labels
  train_sent = clean(train_sent)

  tokenizer = Tokenizer(num_words=None, lower=True, oov_token='<UNK>')
  tokenizer.fit_on_texts(train_sent)

  # Tokenize with appropriate max_word length
  tokenizer = Tokenizer(num_words=len(tokenizer.word_counts.items()), lower=True, oov_token='<UNK>')
  tokenizer.fit_on_texts(train_sent)
  train_sent_tok = tokenizer.texts_to_sequences(train_sent)
  vocab = {k: v for k, v in tokenizer.word_counts.items() if v >= 1}
  vocab_len = len(vocab)

  # Max length in the SST training set was 50 
  train_sent_tok = pad_sequences(train_sent_tok, maxlen=CNN_MAX_LENGTH, truncating='post',
                            padding='post', value=0)

  i2w = {v: k for k, v in tokenizer.word_index.items()}
  # Create Glove Embedding dictionary
  glove_embedding_matrix = create_embedding_indexmatrix(vocab_len, 
                                                        embedding_dim=300,
                                                        dict_en=i2w)
  
  


In [0]:

# Get the apropriate EMBEDDING based on which one is training on
embeddings = []
CNN_X = []
if EMBEDDING == "RANDOM":
  CNN_X = train_sent_tok
elif EMBEDDING == "ELMO":
  embeddingDir = os.path.join(elmoDir)
  embedding_file = os.path.join(embeddingDir, 'train_elmo_embeddings.hdf5')
  print()
  with h5py.File(embedding_file, 'r') as fin:
      for i in range(len(sent_labels)):
        embedding = fin[str(i)][...]     
        embedding = embedding.reshape(embedding.shape[1], embedding.shape[0]*embedding.shape[2])
        if embedding.shape[0] > CNN_MAX_LENGTH:
          CNN_MAX_LENGTH=embedding.shape[0]
        embeddings.append(embedding)
  for embedding in embeddings:
    padding = np.zeros((CNN_MAX_LENGTH - embedding.shape[0], embedding.shape[1]))
    embedding = np.append(embedding, padding, axis=0)
    CNN_X.append(embedding)
elif EMBEDDING == 'COVE':
  ENCODING_PATH = './encoding/' + ENCODER_MODEL + '/' + str(RUN_NUMBER) +'/'
  with open (ENCODING_PATH +CNN_DATASET+ 'encodings', 'rb') as fp:
    encodings = pickle.load(fp)
  CNN_X = encodings
elif EMBEDDING == 'FRENCH':
  ENCODING_PATH = './encoding/' + "FRECH-" + ENCODER_MODEL + '/' + str(RUN_NUMBER) +'/'
  with open (ENCODING_PATH +CNN_DATASET+ 'encodings', 'rb') as fp:
    encodings = pickle.load(fp)
  CNN_X = encodings
elif EMBEDDING == 'LARGE':
  ENCODING_PATH = './encoding/' + "LARGE-"+ENCODER_MODEL + '/' + str(RUN_NUMBER) +'/'
  with open (ENCODING_PATH +CNN_DATASET+ 'encodings', 'rb') as fp:
    encodings = pickle.load(fp)
  CNN_X = encodings
elif EMBEDDING == 'COVE+CHAR':
  !pip install git+https://github.com/vzhong/embeddings.git
  from embeddings import GloveEmbedding, FastTextEmbedding, KazumaCharEmbedding, ConcatEmbedding
  k = KazumaCharEmbedding()
  zeros = np.zeros((100))
  char_embeds = []
  encodings = []
  ENCODING_PATH = './encoding/' + ENCODER_MODEL + '/' + str(RUN_NUMBER) +'/'
  with open (ENCODING_PATH +CNN_DATASET+ 'encodings', 'rb') as fp:
    encodings = pickle.load(fp)
  for i in train_sent:
    char_embed = []
    for n in i.split(" "):
      char_embed.append(k.emb(n))
    while len(char_embed) < CNN_MAX_LENGTH:
      char_embed.append(zeros)
    char_embeds.append(char_embed)
  embeddings = []
  for i in range(0,len(char_embeds)):
    sentence_embedding = []
    for n in range(0, len(char_embeds[i])):
      l = np.concatenate([encodings[i][n],char_embeds[i][n]], axis = 0)
      sentence_embedding.append(l)
    print(i)
    embeddings.append(sentence_embedding)
  CNN_X = embeddings
elif EMBEDDING == "GLOVE" or EMBEDDING == "CHAR+GLOVE":
  # Get Glove embeddings and char embeddings
  zeros = np.zeros((100))
  char_embeds = []
  glove_embeds =[]
  for i in train_sent_tok:
    embedding= []
    for n in i:
      embedding.append(glove_embedding_matrix[n])  
    glove_embeds.append(embedding)
  if EMBEDDING == "CHAR+GLOVE":
    !pip install git+https://github.com/vzhong/embeddings.git
    from embeddings import GloveEmbedding, FastTextEmbedding, KazumaCharEmbedding, ConcatEmbedding
    k = KazumaCharEmbedding()
    for i in train_sent:
      char_embed = []
      for n in i.split(" "):
        char_embed.append(k.emb(n))
      while len(char_embed) < CNN_MAX_LENGTH:
        char_embed.append(zeros)
      char_embeds.append(char_embed)
    for i in range(0,len(glove_embeds)):
      sentence_embed = []
      for n in range(0, len(glove_embeds[0])):
        l = np.concatenate((glove_embeds[i][n],char_embeds[i][n]),axis = 0)
        sentence_embed.append(l)
      embeddings.append(sentence_embed)
    CNN_X = embeddings
  elif EMBEDDING == "GLOVE":
    CNN_X = glove_embeds
elif EMBEDDING == 'BASELINE':
  ENCODING_PATH = './encoding/' + "BASELINE" + '/' + str(2) +'/'
  with open (ENCODING_PATH +CNN_DATASET+ 'train_encodings', 'rb') as fp:
    encodings = pickle.load(fp)
  CNN_X = encodings

Here


In [0]:
# Lambda layer for matrix multiplication
def multiply_t_a(Z):
  return tf.linalg.matmul(Z[0], Z[1], transpose_a=True)

def multiply_t_b(Z):
  return tf.linalg.matmul(Z[0], Z[1], transpose_b=True)

def multiply_no_t(Z):
  return tf.linalg.matmul(Z[0], Z[1])

# Lambda layers for maxout network
def maxout_layer1(Z):
  return tf.contrib.layers.maxout(num_units = FINAL_DIM, inputs = Z)

# Lambda layers for maxout network
def maxout_layer2(Z):
  return tf.contrib.layers.maxout(num_units = int(FINAL_DIM/2), inputs = Z)

def maxout_layer3(Z):
  return tf.contrib.layers.maxout(num_units = 1, inputs = Z)

def squeeze(Z):
  return tf.keras.backend.squeeze(Z, axis=2)

def max_pooling(Z):
  return tf.reduce_max(Z, axis=1)

def min_pooling(Z):
  return tf.reduce_min(Z, axis=1)

def mean_pooling(Z):
  return tf.reduce_mean(Z, axis=1)

def expand(Z):
  return tf.expand_dims(Z, 1)

In [0]:
# Convert from floats to one-hot vectors
if CNN_DATASET == "SST-5":
  from keras.utils import to_categorical
  bin_test_sent_labels = to_categorical([min(4, math.floor(5*x)) for x in sent_labels], dtype=int)
  print(bin_test_sent_labels)
else:
  from keras.utils import to_categorical
  bin_test_sent_labels= to_categorical([min(1, math.floor(1*x)) for x in sent_labels], dtype=int)
  

## Shuffle data 
c = list(zip(CNN_X,bin_test_sent_labels))
np.random.shuffle(c)
CNN_X_sh,bin_test_sent_labels_sh = zip(*c)
CNN_X_sh = CNN_X_sh[:len(CNN_X_sh)]
train_size = int(len(CNN_X_sh)*.8)
val_size = int((len(CNN_X_sh) -train_size)*.5)

# Train split ~ 8/10
CNN_X_train = CNN_X_sh[:train_size]
train_sent_labels = bin_test_sent_labels_sh[:train_size]

# Test split ~ 2/10
CNN_X_test = CNN_X_sh[train_size:]
test_sent_labels = bin_test_sent_labels_sh[train_size:]


CNN_X_test = CNN_X_sh[train_size:train_size+val_size]
test_sent_labels = bin_test_sent_labels_sh[train_size:train_size+val_size]
CNN_X_val = CNN_X_sh[train_size+val_size:]
val_sent_labels = bin_test_sent_labels_sh[train_size+val_size:]
BATCHES = int(math.ceil(len(CNN_X_train)/BATCH_SIZE))

NameError: ignored

In [0]:

# Build Model
model_input = Input(shape=(CNN_MAX_LENGTH,CNN_DIM))


### Commented out code for Rando
# w_x_embed = Embedding(input_dim=vocab_len, output_dim=CNN_DIM,
#                                  embeddings_initializer='RandomNormal',
#                                  input_length=CNN_MAX_LENGTH,
#                                  trainable=False) (model_input)



### 1- LAYER MODEL ###
# w_x_drop = Dropout(rate=.5)(model_input)

# convolutions = []
# filter_sizes = (3,4,5)
# for size in filter_sizes:
#     conv = Conv1D(filters=128,
#                          kernel_size=size,
#                          kernel_constraint=tf.keras.constraints.MaxNorm(max_value=2),
#                          padding="same",
#                          activation="relu",
#                          strides=1)(w_x_drop)
#     conv_pooled = MaxPooling1D(pool_size=2)(conv)
#     conv_flat = Flatten()(conv_pooled)
#     convolutions.append(conv_flat)

# concat_conv = Concatenate()(convolutions) 
# conv_drop = Dropout(rate=.8)(concat_conv)

# result_1 = Dense(1024, activation="relu")(conv_drop)

# result = Dense(N_TARGET,activation="softmax") (result_1)



### 3- LAYER MODEL ###
w_x_drop = Dropout(rate=.5)(model_input)
convolutions_1 = []
filter_sizes = (3,4,5)
for size in filter_sizes:
    conv = Conv1D(filters=256,
                         kernel_size=size,
                         kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3),
                         padding="same",
                         activation="relu",
                         strides=1)(w_x_drop)
    conv_pooled = MaxPooling1D(pool_size=2)(conv)
    convolutions_1.append(conv_pooled)

concat_conv_1 = Concatenate(axis=1)(convolutions_1) 
w_x_drop_2 = Dropout(rate=.5)(concat_conv_1)
print(w_x_drop_2)
convolutions_2 = []
filter_sizes = (4,5,6)
for size in filter_sizes:
    conv = Conv1D(filters=128,
                         kernel_size=size,
                         kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3),
                         padding="same",
                         activation="relu",
                         strides=1)(w_x_drop_2)
    conv_pooled = MaxPooling1D(pool_size=2)(conv)
    convolutions_2.append(conv_pooled)

concat_conv_2 = Concatenate(axis=1)(convolutions_2) 
w_x_drop_3 = Dropout(rate=.5)(concat_conv_2)

convolutions_3 = []
filter_sizes = (4,5,6)

for size in filter_sizes:
    conv = Conv1D(filters=128,
                         kernel_size=size,
                         kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3),
                         padding="same",
                         activation="relu",
                         strides=1)(w_x_drop_3)
    conv_pooled = MaxPooling1D(pool_size=2)(conv)
    conv_flat = Flatten()(conv_pooled)
    convolutions_3.append(conv_flat)

concat_conv_3 = Concatenate(axis=1)(convolutions_3) 


conv_drop = Dropout(rate=.5)(concat_conv_3)
result_1 = Dense(1024, activation="relu")(conv_drop)
result = Dense(N_TARGET,activation="softmax") (result_1)
CNN = Model(model_input,result)


opt = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, clipnorm=.00005, epsilon=None, decay=0.0, amsgrad=False)
CNN.compile(optimizer = opt, loss = 'categorical_crossentropy',metrics=["accuracy"])
CNN.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Colocations handled automatically by placer.
Tensor("dropout_1/cond/Merge:0", shape=(?, 51, 256), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 35, 900)      0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 35, 900)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 35, 256)      691456      dropout[0][0]                    
____________________________________________________

In [0]:
# Testing
def test():
  CNN_pred = CNN.predict(x=[CNN_X_test])
  acc = 0
  if N_TARGET == 5:
    for i in range(len(test_sent_labels)):
      acc += int(test_sent_labels[i].argmax(axis=0) == CNN_pred[i].argmax(axis=0))
  elif N_TARGET ==2:
    for i in range(len(test_sent_labels)):
       acc += int(test_sent_labels[i].argmax(axis=0) == CNN_pred[i].argmax(axis=0))

  print(acc/len(test_sent_labels))
  return acc/len(test_sent_labels)

In [0]:
# Testing
def val():
  CNN_pred = CNN.predict(x=[CNN_X_val,CNN_X_val])
  acc = 0
  if N_TARGET == 5:
    for i in range(len(val_sent_labels)):
      acc += int(val_sent_labels[i].argmax(axis=0) == CNN_pred[i].argmax(axis=0))
  elif N_TARGET ==2:
    for i in range(len(val_sent_labels)):
       acc += int(val_sent_labels[i].argmax(axis=0) == CNN_pred[i].argmax(axis=0))

  print(acc/len(val_sent_labels))
  return acc/len(val_sent_labels)

In [0]:
# Training
BATCH_SIZE = 256
[sec_last_loss, last_loss, lowest_val_loss] = [5000, 5000 , 5000]
delta = 0
BATCHES = int(math.ceil(len(CNN_X_train)/BATCH_SIZE))
losses = []
test_acc = 0 
for j in range(100):
  print("EPOCH",j)
  losses = []
  start = time.time()
  for i in range(0,BATCHES):
    output = CNN.train_on_batch([CNN_X_train[i*BATCH_SIZE:BATCH_SIZE+i*BATCH_SIZE]],[train_sent_labels[i*BATCH_SIZE:BATCH_SIZE+i*BATCH_SIZE]]) 
    losses.append(output[0])
  val_loss, val_acc= CNN.test_on_batch(x=[CNN_X_val,
                                   CNN_X_val],
                                y=[val_sent_labels]) 
  print(val_loss)
  print(val_acc)

  if last_loss - val_loss < delta and sec_last_loss - last_loss < delta:
      break
  else:
      sec_last_loss = last_loss
      last_loss = val_loss 


EPOCH 0
Instructions for updating:
Use tf.cast instead.
0.69616747
0.4898844
0.45231214
EPOCH 1
0.68747514
0.57225436
0.57947975
EPOCH 2
0.685331
0.55057806
0.5910405
EPOCH 3
0.6812582
0.53757226
0.55924857
EPOCH 4
0.67327994
0.5433526
0.5693642
EPOCH 5
0.6499332
0.6083815
0.6213873
EPOCH 6
0.6230341
0.61849713
0.63439304
EPOCH 7
0.586125
0.68352604
0.6965318
EPOCH 8
0.5694194
0.716763
0.7080925
EPOCH 9
0.56167954
0.72254336
0.7080925
EPOCH 10
0.5409721
0.7384393
0.7456647
EPOCH 11
0.52036405
0.7586705
0.7890173
EPOCH 12
0.51863784
0.75722545
0.7731214
EPOCH 13
0.5103321
0.7615607
0.7817919
EPOCH 14
0.5126021
0.7601156
0.7745665
EPOCH 15
0.49894354
0.7731214
0.783237
EPOCH 16
0.49654877
0.7731214
0.79046243
EPOCH 17
0.5003568
0.75722545
0.7731214
EPOCH 18
0.49652293
0.76589596
0.77745664
EPOCH 19
0.48539245
0.7731214
0.79913294
EPOCH 20
0.4751735
0.7745665
0.79046243
EPOCH 21
0.4777852
0.767341
0.77890176
EPOCH 22
0.46425998
0.7976879
0.8049133
EPOCH 23
0.4682727
0.8034682
0.7976879
EP

In [0]:
CNN_json = CNN.to_json()
train_CNN_json = os.path.join(elmoDir, 'CNN.json')
train_CNN_h5 = os.path.join(elmoDir, 'CNN.h5')
with open(train_CNN_json, "w") as json_file:
    json_file.write(CNN_json)
# serialize weights to HDF5
CNN.save_weights(train_CNN_h5)
print("Saved model to disk")

Saved model to disk
